In [2]:
import pandas as pd
import scipy.stats
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shap

### Methods that work for both classification and regression (all types of features)

In [ ]:
clf_important_features = pd.DataFrame(index = features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=5, n_estimators=10)
rf.fit(train_x, train_y)

clf_important_features.loc[features, 'RandomForestClf'] = rf.feature_importances_

In [ ]:
from sklearn.neural_network import MLPClassifier
   
clf = MLPClassifier(alpha=1, max_iter=1000)
clf.fit(train_x, train_y)

explainer_clf = shap.KernelExplainer(clf.predict, data_df)
shap_values_clf = explainer_clf.shap_values(data_df)

In [ ]:
shap.summary_plot(shap_values = shap_values_clf, features = train_x, 
                  feature_names = important_features, class_names = ['not detected', 'detected'], 
                  color_bar_label = 'Feature level',
                  max_display=40,
                  show=False
                )
plt.tight_layout()

In [ ]:
# find which feature contributes in a positive or negative way

shap_values_clf_df = pd.DataFrame(shap_values_clf, columns = features, index = observations).T
clf_signs = []
for i, row in shap_values_clf_df.iterrows():
    c = np.where(row < 0)
    neg_patients = shap_values_clf_df.columns[c].tolist()
    neg_val = features_norm_df.loc[neg_patients][i].mean()
    c = np.where(row >= 0)
    pos_patients = shap_values_clf_df.columns[c].tolist()
    pos_val = features_norm_df.loc[pos_patients][i].mean()
    if neg_val >= pos_val: sign = -1
    else: sign = 1
    clf_signs.append(sign)

clf_important_features.loc[features, 'shap_feature_importance_abs_mean_sign'] = np.abs(shap_values_clf).mean(axis=0) * clf_signs

### Additional methods for regression only (not good for binary features)

In [ ]:
# Mutual information
fi_clf = mutual_info_regression(train_x, train_y)
reg_important_features['mutual_info'] = fi_clf

# Polynomial fit
polifit_res = np.polyfit(drug_rows['viability'].values, features_norm_df.drop(columns = 'patient'), 
                          2, full=True)
reg_important_features['polyfit'] = polifit_res[1]

# Correlation
pearson = features_df.corrwith(intensities, method = 'pearson')
spearman = features_df.corrwith(intensities, method = 'spearman')
reg_important_features['pearson'] = pearson.values
reg_important_features['spearman'] = spearman.values

# Linear regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_x, train_y)
important_features_df['Lin reg importance'] = lr.coef_[0]

### For each importance criteria, test several thresholds on feature importance

In [ ]:
for metric in important_features_df.columns:
        metric_important_features_df = important_features_df[metric]
        v_min = metric_important_features_df.abs().min()
        v_max = metric_important_features_df.abs().max()
        n_thresholds = 5 # NUMBER OF THRESHOLDS, CAN BE CHANGED
        thresholds = np.linspace(v_min, v_max, n_thresholds+2)[1:-1]
               
        for t in thresholds:
            important_features = metric_important_features_df[metric_important_features_df.abs()>t].index.tolist()
            if len(important_features) == 0: continue
            threshold_results_df = train_test_models(...)